In [1]:
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import math
import warnings

from sklearn.semi_supervised import LabelPropagation
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.neighbors import kneighbors_graph
from sklearn.neighbors import NearestNeighbors

from scipy.sparse import triu
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
from scipy.sparse import csgraph
from scipy.sparse import diags
from scipy import sparse

from sklearn.base import BaseEstimator, ClassifierMixin

from sklearn.utils.multiclass import check_classification_targets

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import ConvergenceWarning
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.neighbors import NearestNeighbors
from sklearn.utils._param_validation import Interval, StrOptions
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.utils.multiclass import check_classification_targets
from sklearn.utils.validation import check_is_fitted

from collections import Counter

from ast import literal_eval
from gensim.matutils import corpus2csc
import scipy.sparse as scsp

In [2]:
df = pd.read_parquet("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/models/POLI_FILTERED_AL/rosie_1_20/df_graph_en.parquet")

In [3]:
len(df)

985225

In [3]:
df[df.thetas == "[(0, nan), (1, nan), (2, nan), (3, nan), (4, nan), (5, nan), (6, nan), (7, nan), (8, nan), (9, nan), (10, nan), (11, nan), (12, nan), (13, nan), (14, nan), (15, nan), (16, nan), (17, nan), (18, nan), (19, nan)]"]

,id_preproc,lemmas,lemmas_tr,doc_id,text,text_tr,lang,len,thetas,top_k,main_topic,label
853945,853945,kevin_flanigan tolerance gene,kevin_flanigan tolerancia gen,EN_1417065_305900-60,Kevin Flanigan: They've already get tolerance ...,Kevin Flanigan: Ya tienen tolerancia por ese g...,EN,3,"[(0, nan), (1, nan), (2, nan), (3, nan), (4, n...","[[1.0, nan], [2.0, nan]]",1,Cardiovascular Health Research and Prevention


In [4]:
df = df.drop(index=853945)

In [5]:
thetas =  sparse.load_npz("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/models/POLI_FILTERED_AL/rosie_1_20/mallet_output/thetas_EN.npz").toarray()

In [6]:
corpus_size = len(df)

df_sample = df
sample_factor = 1
n_docs = corpus_size

print("-- - Sample documents:")
print(df_sample.head())

X = [literal_eval(el) for el in df_sample['thetas'].values.tolist()]
X = corpus2csc(X).T
n_topics = X.shape[1]
print(f"-- -- Number of topics: {n_topics}")
print(f"X: sparse matrix with {X.nnz} nonzero values out of {n_docs * n_topics}")
print(X.shape)

# Normalization
X = scsp.csr_matrix(X / np.sum(X, axis=1))
print(f"-- -- Average row sum: {np.mean(X.sum(axis=1).T)}")

#S = np.sqrt(X) * np.sqrt(X.T)
#print("-- -- Number of non-zero components: ", S.nnz)
#nnz_prop = S.nnz / (S.shape[0] * S.shape[1])

-- - Sample documents:
   id_preproc                                             lemmas  \
0           0  implant_port information technology easy child...   
1           1  life_expectancy birth reach record high year t...   
2           2  shazam_hussain arterial common location caroti...   
3           3  rifaximin cause effect doctor unusual_problem ...   
4           4  point healing_process need dressing provider l...   

                                           lemmas_tr                doc_id  \
0  puerto_implantado facilitar hijo recibir trata...   EN_657272_107210-27   
1  esperanza_vida nacer alcanzar_máximo precedent...     EN_403516_60988-2   
2  shazam_hussain arterial común provenir arteria...   EN_736110_112440-10   
3  rifaximina causar secundario_llamir médico pro...  EN_1279033_283895-10   
4  curación necesitar apósito proveedor dejar her...  EN_1241536_280479-12   

                                                text  \
0  Implanted ports make it easier for you o

In [7]:
class CustomLabelPropagation(LabelPropagation):

    def __init__(self, threshold, symmetric=False, use_connectivity=False, sin_autoenlaces=False, **kwargs):
        self.threshold = threshold
        self.nn_fit = None
        self.symmetric = symmetric
        self.use_connectivity = use_connectivity
        self.sin_autoenlaces = sin_autoenlaces
        self.graph_stats_ = None
        super().__init__(**kwargs)

    def _get_kernel(self, X, y=None):
        if self.kernel == "umbral" or self.kernel == "hibrido":
            if y is None:
                # Compute the cosine distance
                return np.sqrt(X) * np.sqrt(X.T)
            else:
                return 1 - cosine_distances(X, y)

        elif self.kernel == "knn":
            # Metiendo scikit-learn
            if self.nn_fit is None:
                # print("esta haciendo el NearestNeighbors")
                self.nn_fit = NearestNeighbors(
                    n_neighbors=self.n_neighbors, n_jobs=self.n_jobs, metric='cosine'
                ).fit(X)
            if y is None:
                if self.use_connectivity:
                    return self.nn_fit.kneighbors_graph(
                        self.nn_fit._fit_X, self.n_neighbors, mode="connectivity"
                    )
                else:
                    # Usar matriz de similitudes
                    similarity_matrix = self.nn_fit.kneighbors_graph(
                        self.nn_fit._fit_X, self.n_neighbors, mode='distance',
                    )
                    similarity_matrix.data = 1 - similarity_matrix.data
                    return similarity_matrix

            else:
                # Aquí también entra es para devolver los índices de los vecinos que devuelve
                return self.nn_fit.kneighbors(y, return_distance=False)

        elif self.kernel == "rbf":
            if y is None:
                return rbf_kernel(X, X, gamma=self.gamma)

        elif callable(self.kernel):
            if y is None:
                return self.kernel(X, X)
            else:
                return self.kernel(X, y)
            
        print("KERNEL CALCULATED")

    @staticmethod
    def make_symmetric_csr(matrix):
        # Hacer simétrica
        matrix = matrix + matrix.T
        matrix.data[matrix.data > 1] = 1
        return matrix

    def combine_matrices(self, knn_matrix, threshold_matrix, max_links):
        num_nodes = knn_matrix.shape[0]
        combined_matrix = np.zeros_like(threshold_matrix)

        # Convertir solo la matriz KNN dispersa a una matriz densa
        dense_knn = knn_matrix.toarray()

        for i in range(num_nodes):
            # Combinar enlaces de k-NN y umbral
            all_links = np.argsort(-np.maximum(dense_knn[i], threshold_matrix[i]))[:max_links]

            # Asignar los enlaces más fuertes a la matriz combinada
            for link in all_links:
                combined_matrix[i, link] = max(dense_knn[i, link], threshold_matrix[i, link])

        return combined_matrix

    def _build_graph(self, X):
        self.X_ = X

        # Si el kernel es "umbral", usa el _build_graph de LabelPropagation
        if self.kernel == "knn":
            # print("Si es knn")
            self.graph_ = self._get_kernel(self.X_)
            # Eliminar la diagonal principal, para eliminar autoenlaces
            if self.sin_autoenlaces:
                self.graph_.setdiag(0)
                # self.graph_.eliminate_zeros()
                # print("IMPRIME la matriz SIN AUTOENLACES:\n", self.graph_)
            if self.symmetric:
                self.graph_ = self.make_symmetric_csr(self.graph_)

            return self.graph_

        elif self.kernel == "umbral":
            # Si el kernel es "umbral", coge la similarity_matrix del get_kernel sin threshold
            self.graph_ = self._get_kernel(self.X_)
            # print("Esta es la matriz del get_kernel, calculada con la similitud del coseno:\n", similarity_matrix)
            # Aplicar el THRESHOLD
            self.graph_[self.graph_ < self.threshold] = 0
            # print("Esta es la matriz con THRESHOLD:\n", similarity_matrix)
            # Guardo la matriz de similitudes en formato sparse para ahorrar espacio en memoria
            self.graph_ = csr_matrix(self.graph_)
            # print("Esta es la matriz SPARSE:\n", similarity_matrix)
            if self.sin_autoenlaces:
                # Eliminar la diagonal principal, para eliminar autoenlaces
                self.graph_.setdiag(0)
                # self.graph_.eliminate_zeros()

            # print("LA MATRIZ SPARSE NORMALIZADA que sale de build_graph:\n", self.graph_)
            return self.graph_

        # Lógica para el kernel "hibrido"
        else:
            # Asegurar que NearestNeighbors está inicializado
            if self.nn_fit is None:
                self.nn_fit = NearestNeighbors(n_neighbors=3, n_jobs=self.n_jobs, metric='cosine').fit(X)  # Usamos 3 para k-NN

            # Obtener la matriz k-NN como similitudes
            knn_matrix = self.nn_fit.kneighbors_graph(self.nn_fit._fit_X, mode='distance')
            knn_matrix.data = 1 - knn_matrix.data  # Convertir distancia a similitud

            # Obtener la matriz completa de similitudes del coseno
            full_cosine_sim_matrix = self._get_kernel(self.X_)

            # Aplicar el umbral
            full_cosine_sim_matrix[full_cosine_sim_matrix < self.threshold] = 0

            # Combinar las matrices de manera que cada nodo tenga un número fijo de enlaces
            combined_matrix = self.combine_matrices(knn_matrix, full_cosine_sim_matrix, max_links=5000)

            # Guardar la matriz combinada en formato sparse
            self.graph_ = csr_matrix(combined_matrix)

            # print("LA Matriz combinada es:", self.graph_)

            # Opcional: Eliminar autoenlaces si es necesario
            if self.sin_autoenlaces:
                self.graph_.setdiag(0)

            return self.graph_

    def visualize_graph(self, X=None, labels=None, similarity_matrix=None, plot_graph=False):
        # Si no se pasa una matriz X, se utilizará self.X_
        if X is None:
            X = self.X_
        # Esto solo ocurre si al llamarla en el fit no le pasara la graph_matrix [no debería entrar NUNCA]
        if similarity_matrix is None:
            # print("NO DEBE ENTRAR NUNCA")
            similarity_matrix = self._build_graph(X, symmetric=self.symmetric)

        # Crea un grafo
        G = nx.DiGraph()

        # Añadir todos los nodos al grafo inicialmente
        for i in range(X.shape[0]):
            G.add_node(i)

        color_nodos = {
            0: 'purple',  # color para clase 0
            1: 'green',   # color para clase 1
            2: 'orange',  # color para clase 2
            3: 'blue',    # color para clase 3
            4: 'pink',    # color para clase 4
            5: 'yellow',  # color para clase 5
            -1: 'gray'    # color para nodos sin etiquetar
        }

        if self.kernel == "knn" or self.kernel == "umbral" or self.kernel == "hibrido":
            # Añade los bordes al grafo usando los datos de la matriz CSR
            for i, j, v in zip(similarity_matrix.nonzero()[0], similarity_matrix.nonzero()[1], similarity_matrix.data):
                G.add_edge(i, j, weight=v)

        # Recopilar estadísticas del grafo
        num_nodes = G.number_of_nodes()
        num_edges = G.number_of_edges()
        edges_per_node = num_edges / num_nodes

        # Determinar el tipo de kernel para el título
        kernel_type = "Similitud" if not self.use_connectivity else "Conectividad"

        # Visualizar el grafo solo si plot_graph es True
        if plot_graph:
            # Dibuja el grafo usando spring_layout
            pos = nx.spring_layout(G, seed=42)

            # Filtrar las etiquetas para nodos en el grafo
            nodes_in_graph = set(G.nodes())
            filtered_labels = [labels[i] for i in nodes_in_graph]

            # Colorear nodos según sus etiquetas
            color_map = [color_nodos[label] for label in filtered_labels]
            nx.draw(G, pos, with_labels=True, node_size=100, node_color=color_map, width=0.2)

            # Imprimir información sobre el grafo
            print(f"Número de nodos: {num_nodes}")
            print(f"Número de enlaces: {num_edges}")
            print(f"Número de enlaces por nodo: {edges_per_node:.2f}")

            plt.title(f"Visualización del Grafo {self.kernel.upper()} - {kernel_type}")
            plt.show()

        # Devuelve las estadísticas del grafo en un diccionario
        graph_stats = {
            'number_of_nodes': num_nodes,
            'number_of_edges': num_edges,
            'edges_per_node': edges_per_node
        }

        # Devuelve las estadísticas
        return graph_stats

    def identify_isolated_nodes(self, graph):
        isolated = []
        for i in range(graph.shape[0]):
            if not graph[i].nonzero()[1].size:  # No hay conexiones
                isolated.append(i)
        return isolated


In [8]:
# Listas para almacenar los valores máximos, mínimos, y el valor de n_neighbors para cada threshold
max_values = []
min_values = []
n_neighbors_values = []

# Instancia de la clase CustomLabelPropagation con el kernel "knn"
clp = CustomLabelPropagation(kernel="umbral", n_neighbors=1, threshold=1, use_connectivity = False)
similarity_matrix = clp._build_graph(X)
print(similarity_matrix.shape)

'''
# Usaremos la función `flatten()` para convertir la matriz en un array 1D
# Convertir la matriz dispersa a una matriz densa y luego aplanarla
flattened_values = similarity_matrix.toarray().flatten()


# Encontrar y almacenar los valores máximo y mínimo en la matrizde similitud, y el valor de n_neighbors
max_values.append(np.max(flattened_values))
min_values.append(np.min(flattened_values))

# Crear un DataFrame con los resultados
result_df = pd.DataFrame({
    'Max Value': max_values,
    'Min Value': min_values
})

print(result_df)
'''

MemoryError: Unable to allocate 6.72 TiB for an array with shape (923705709420,) and data type int64